In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split

# Step 1: Load the dataset
ds = tfds.load('aflw2k3d', split='train')

# Step 2: Extract images and landmarks
images = []
landmarks2D = []

# Extract images and landmarks into separate lists
for ex in ds.take(2000):
    images.append(ex['image'])
    # Extract only the (x, y) coordinates from the landmarks
    landmarks2D.append(ex['landmarks_68_3d_xy_normalized'])

# Convert lists to numpy arrays
images = np.array(images)
landmarks2D = np.array(landmarks2D)

# Flatten landmarks to match the output of the model
landmarks_flat = landmarks2D.reshape(-1, 68 * 2)

# Step 3: Split the dataset into training and testing sets
train_images, test_images, train_landmarks, test_landmarks = train_test_split(images, landmarks_flat, test_size=500, random_state=42)

# Step 4: Build the CNN model
model = tf.keras.Sequential([
    # Your convolutional layers here
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(450, 450, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(68*2)  # Output layer for (x, y) coordinates
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 5: Train the model
model.fit(train_images, train_landmarks, epochs=10, batch_size=32, validation_split=0.1)

# Step 6: Evaluate the model
test_loss = model.evaluate(test_images, test_landmarks)
print("Test Loss:", test_loss)
print("Accuarcy: ",100-test_loss,"%")



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/2000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/aflw2k3d/1.0.0.incompleteVO7PZL/aflw2k3d-train.tfrecord*...:   0%|        …

Dataset aflw2k3d downloaded and prepared to /root/tensorflow_datasets/aflw2k3d/1.0.0. Subsequent calls will reuse this data.
Epoch 1/10
43/43 [==============================] - 84s 2s/step - loss: 19031.1328 - val_loss: 0.3221
Epoch 2/10
43/43 [==============================] - 83s 2s/step - loss: 0.3013 - val_loss: 0.2380
Epoch 3/10
43/43 [==============================] - 82s 2s/step - loss: 0.2137 - val_loss: 0.1586
Epoch 4/10
43/43 [==============================] - 82s 2s/step - loss: 0.1139 - val_loss: 0.1145
Epoch 5/10
43/43 [==============================] - 82s 2s/step - loss: 0.0646 - val_loss: 0.0872
Epoch 6/10
43/43 [==============================] - 81s 2s/step - loss: 0.0398 - val_loss: 0.0779
Epoch 7/10
43/43 [==============================] - 81s 2s/step - loss: 0.0464 - val_loss: 0.0967
Epoch 8/10
43/43 [==============================] - 82s 2s/step - loss: 0.0403 - val_loss: 0.0885
Epoch 9/10
43/43 [==============================] - 82s 2s/step - loss: 0.0260 - val_lo

In [ ]:
# Step 7: Evaluate the model on test data
predictions = model.predict(test_images)

# Calculate Mean Squared Error (MSE)
mse = np.mean(np.square(predictions - test_landmarks))

print("Mean Squared Error (MSE):", mse)

# Calculate Mean Absolute Error (MAE)
mae = np.mean(np.abs(predictions - test_landmarks))

print("Mean Absolute Error (MAE):", mae)

# Define a threshold for correctness
epsilon = 5  # Adjust as needed

# Calculate distance between predicted and ground truth landmarks
distances = np.sqrt(np.sum(np.square(predictions - test_landmarks), axis=1))
correct_predictions = np.sum(distances < epsilon)

accuracy = correct_predictions / len(test_landmarks) * 100

print("Accuracy:", accuracy, "%")

16/16 [==============================] - 5s 330ms/step
Mean Squared Error (MSE): 0.10092585
Mean Absolute Error (MAE): 0.24394266
Accuracy: 86.8 %
